Sketchbook to experiment with RNN models to model the dynamics. Once tested, the code in this notebook will be incorporated into functions/classes/ python modules.

## Dynamics Model
Models the dynamics of the system p(next observation | history of observations, action)

History of observations consist of exercises a student has done and whether the student solved each of them

Action is the next exercise chosen 

Next observation is whether the student gets the chosen exercise correct


We want to use an RNN to model the dynamics.
Input data represents history of observations, of shape (n_students, n_timesteps, observation_vec_size)

Output represents the probability of getting next exercise correctly, of shape (n_students, n_timesteps, n_exercises) 

So at each timestep, we make a prediction for all actions. 

For each action, the output vector specifies the predicted probability of the student getting the chosen exercise correctly.

The target output only contains binary values.

In [1]:
import sys
print sys.executable
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

/Users/lisa1010/tf_venv/bin/python


In [2]:
import sonnet as snt
import tensorflow as tf
import tflearn
import numpy as np

In [3]:
import dataset_utils

Load toy data set

In [5]:
data = dataset_utils.load_data(filename="../synthetic_data/toy.pickle")
input_data_, output_mask_, target_data_ = dataset_utils.preprocess_data_for_rnn(data)

# Build RNN Model

In [6]:
tflearn.init_graph()

gpu_options {
}
allow_soft_placement: true

In [9]:
n_hidden = 64
n_samples, n_timesteps, n_inputdim = input_data_.shape
_,_,n_outputdim = target_data_.shape

In [10]:
print n_timesteps

50


In [12]:
print n_inputdim

20


In [13]:
print n_outputdim

10


In [15]:
graph_to_use = tf.Graph()
with graph_to_use.as_default():
    net = tflearn.input_data([None, n_timesteps, n_inputdim],dtype=tf.float32, name='input_data')
    output_mask = tflearn.input_data([None, n_timesteps, n_outputdim], dtype=tf.float32, name='output_mask')
    net = tflearn.lstm(net, n_hidden, return_seq=True, name="lstm_1")
    net = tflearn.lstm(net, n_outputdim, return_seq=True, name="lstm_2")
    net = tf.stack(net, axis=1)
    preds = net
    net = net * output_mask
    net = tflearn.regression(net, optimizer='adam', learning_rate=0.001,
                             loss='mean_square')
    model = tflearn.DNN(net, tensorboard_verbose=0)
    model.fit([ input_data_, output_mask_], target_data_, validation_set=0.1)

---------------------------------
Run id: 2PTSBL
Log directory: /tmp/tflearn_logs/
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'list' object has no attribute 'name'
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'list' object has no attribute 'name'
---------------------------------
Training samples: 4
Validation samples: 1
--
Training Step: 1  | time: 2.116s
| Adam | epoch: 001 | loss: 0.00000 | val_loss: 0.04748 -- iter: 4/4
--
Training Step: 2  | total loss: 0.04031 | time: 1.067s
| Adam | epoch: 002 | loss: 0.04031 | val_loss: 0.04669 -- iter: 4/4
--
Training Step: 3  | total loss: 0.04343 | time: 1.065s
| Adam | epoch: 003 | loss: 0.04343 | val_loss: 0.04589 -- iter: 4/4
--
Training Step: 4  | total loss: 0.04341 | time: 1.061s
| Adam | epoch: 004 | loss: 0.04341 | val_loss: 0.04506 -- iter: 4/4
--
Training Step: 5  | total loss: 0.04293 | time: 1.060s
| Adam | epoch: 005 | loss: 0.04293 | val_los

In [72]:
tf.get_collection(tf.GraphKeys.INPUTS)

[<tf.Tensor 'output_mask:0' shape=(?, 50, 10) dtype=int32>,
 <tf.Tensor 'InputData/X:0' shape=(?, 50, 20) dtype=float32>,
 <tf.Tensor 'output_mask_1:0' shape=(?, 50, 10) dtype=int32>,
 <tf.Tensor 'InputData_1/X:0' shape=(?, 50, 20) dtype=float32>,
 <tf.Tensor 'output_mask_2:0' shape=(?, 50, 10) dtype=int32>,
 <tf.Tensor 'InputData_2/X:0' shape=(?, 50, 20) dtype=float32>,
 <tf.Tensor 'output_mask_3:0' shape=(?, 50, 10) dtype=float32>,
 <tf.Tensor 'InputData_3/X:0' shape=(?, 50, 20) dtype=float32>,
 <tf.Tensor 'input_data:0' shape=(?, 50, 20) dtype=float32>,
 <tf.Tensor 'output_mask_4:0' shape=(?, 50, 10) dtype=float32>,
 <tf.Tensor 'input_data_1:0' shape=(?, 50, 20) dtype=float32>,
 <tf.Tensor 'output_mask_5:0' shape=(?, 50, 10) dtype=float32>,
 <tf.Tensor 'input_data_2:0' shape=(?, 50, 20) dtype=float32>,
 <tf.Tensor 'output_mask_6:0' shape=(?, 50, 10) dtype=float32>,
 <tf.Tensor 'input_data_3:0' shape=(?, 50, 20) dtype=float32>,
 <tf.Tensor 'output_mask_7:0' shape=(?, 50, 10) dtype=fl